In [ ]:
from algorithm_0 import Algorithm
from disjoint_set_union import RangeDSU
from itertools import product
#from sage.all import *
#from sage.crypto.sboxes import sboxes, monomial_function, SBox
#from sage.crypto.boolean_function import BooleanFunction
#from collections import Counter

In [ ]:
#from queue import PriorityQueue  # too slow
from heapq import heappush, heappop
from random import choice, random as random_ratio, shuffle
from disjoint_set_union import RangeDSU

def GreedyRunHeap(Alg, init=(), force=False, debug=0, profile=False, LB=None):
    n = Alg.n
    m = Alg.m
    S = Alg.S
    
    LS = {0}
    LSx = {0}
    
    #queue = PriorityQueue()

    if profile:
        profile = [1]

    cliques = RangeDSU(2**n)
    if LB is not None:
        cosets = RangeDSU(2**n)
        max_cl = {i:1 for i in range(2**n)}
    #queue = [(-1, i) for i in range(2**n)]
    queue = [(-1, random_ratio(), i) for i in range(2**n)]
    shuffle(queue)
    
    ddt2 = Alg.compute_kddt(k=2)
    
    init = list(init)

    
    
    for i in range(n):
        if debug: print("iteration", i, "\n========")

        cl1 = cl2 = None
        while len(init) >= 2:
            cl1 = cliques.find(init.pop())
            cl2 = cliques.find(init.pop())
            init.append(cl1)
            if cl1 == cl2:
                cl1 = cl2 = None
                continue
            #print("merge init", cl1, cl2)
            break
        else:
            if init and force:
                cl1 = init[0]
            
        if cl1 is None:
            while True:
                sz1, _, cl1 = heappop(queue)
                sz1 = -sz1
                if cliques.size(cl1) == sz1:
                    break
        
        # find max-size clique from another coset
        if cl2 is None:
            tmp = []
            while True:
                sz2, _, cl2 = heappop(queue)
                sz2 = -sz2
                if cliques.size(cl2) != sz2:
                    continue
                tmp.append((-sz2, random_ratio(), cl2))
                dx = cl1 ^ cl2
                if dx not in LSx:
                    break
                    
                # dy = S[cl1] ^ S[cl2]
                # dxdy = (dx << n) | dy
                # if dxdy not in LS:
                #     break
            for q in tmp:
                heappush(queue, q)
        
        if profile:
            profile.append(sz1 + sz2)
    
        # cl identifier is one of the elements itself
        x1 = cl1
        x2 = cl2
        dxdy0 = ((x1 ^ x2) << m) | (S[x1] ^ S[x2])
        dx0 = x1 ^ x2

        LS_new = set(LS)
        # merge cliques
        for dxdy in LS:
            dxdy ^= dxdy0
            LS_new.add(dxdy)
            for x1, x2 in ddt2[dxdy >> m][dxdy & (2**m-1)]:
                cliques.union(x1, x2)
                sz = cliques.size(x1)
                heappush(queue, (-sz, random_ratio(), x1))
                if LB is not None:
                    coset1 = cosets.find(x1)
                    max_cl[coset1] = max(sz,max_cl.get(coset1,0))
              
                
        if LB is not None:
            # merge cosets
            #print(cosets.components())
            sum_max = 0
            for coset in cosets.components():
                #print(coset,coset^dx0)
                coset = cosets.find(coset)
                coset2 = cosets.find(coset ^ dx0)
                if coset != coset2:
                    #print(coset,coset2)
                    sz = max(max_cl[coset],max_cl[coset2])
                    del max_cl[coset]
                    del max_cl[coset2]
                    cosets.union(coset, coset2)
                    #print(coset,coset2,dx0)
                    max_cl[cosets.find(coset)] = sz
                    sum_max += sz
            #print(cosets.list())
                
            #for col in ddt2[dx0]:    # alternatively (but slower)
            #    for x1,x2 in col:
            #        cosets.union(x1,x2) 
    
            #max_cl = {}
            #for cl in cliques._components:
            #    sz = cliques.size(cl)
            #    coset = cosets.find(cl)
            #    max_cl[coset] = max(sz,max_cl.get(coset,0))
            #print(sum_max)
            if sum_max < LB:
                return None
                
        #LS = LS | {v ^ dxdy0 for v in LS}
        LS = LS_new
        LSx = LSx | {v ^ dx0 for v in LSx}
        
    cls = cliques.list()
    mx = max(len(cl) for cl in cls)
    for cl in cls:
        if len(cl) == mx:
            break
    else:
        assert 0
    if profile:
        return cl, profile
    return cl

In [ ]:
import hashlib
import pathlib

def create_file(L,name,n):
    folder_path = "results/"+str(n)+"-bit/"+name
    print(folder_path)
    pathlib.Path(folder_path).mkdir(parents=True,exist_ok=True) 
    L = sorted(L)
    h = hashlib.sha256(str(L).encode()).hexdigest()
    with open(folder_path+"/"+str(len(L))+"_"+h+".txt","w") as f:
        f.write(str(L))

In [ ]:
# Midori
Sb0 = 0xc, 0xa, 0xd, 0x3, 0xe, 0xb, 0xf, 0x7, 0x8, 0x9, 0x1, 0x5, 0x0, 0x2, 0x4, 0x6
#Sb1 = 0x1, 0x0, 0x5, 0x3, 0xe, 0x2, 0xf, 0x7, 0xd, 0xa, 0x9, 0xb, 0xc, 0x8, 0x4, 0x6

n = m = 4

SuperSbox = []
k = 12,12,12,12
for a, b, c, d in product(range(2**n), repeat=4):
    a, b, c, d = [Sb0[x] for x in (a, b, c, d)]    
    t = a ^ b ^ c ^ d
    a ^= t ^ k[0]
    b ^= t ^ k[1]
    c ^= t ^ k[2]
    d ^= t ^ k[3]
    a, b, c, d = [Sb0[x] for x in (a, b, c, d)]
    y = (a << 12) | (b << 8) | (c << 4) | d
    SuperSbox.append(y)

name = "Midori"
for i in range(4):
    name += "_"+str(k[i])
print(name)

In [ ]:
misty_s7 = [0x36, 0x32, 0x3e, 0x38, 0x16, 0x22, 0x5e, 0x60, 0x26, 0x6, 0x3f, 0x5d, 0x2, 0x12, 0x7b, 0x21, 0x37, 0x71, 0x27, 0x72, 0x15, 0x43, 0x41, 0xc, 0x2f, 0x49, 0x2e, 0x1b, 0x19, 0x6f, 0x7c, 0x51, 0x35, 0x9, 0x79, 0x4f, 0x34, 0x3c, 0x3a, 0x30, 0x65, 0x7f, 0x28, 0x78, 0x68, 0x46, 0x47, 0x2b, 0x14, 0x7a, 0x48, 0x3d, 0x17, 0x6d, 0xd, 0x64, 0x4d, 0x1, 0x10, 0x7, 0x52, 0xa, 0x69, 0x62, 0x75, 0x74, 0x4c, 0xb, 0x59, 0x6a, 0x0, 0x7d, 0x76, 0x63, 0x56, 0x45, 0x1e, 0x39, 0x7e, 0x57, 0x70, 0x33, 0x11, 0x5, 0x5f, 0xe, 0x5a, 0x54, 0x5b, 0x8, 0x23, 0x67, 0x20, 0x61, 0x1c, 0x42, 0x66, 0x1f, 0x1a, 0x2d, 0x4b, 0x4, 0x55, 0x5c, 0x25, 0x4a, 0x50, 0x31, 0x44, 0x1d, 0x73, 0x2c, 0x40, 0x6b, 0x6c, 0x18, 0x6e, 0x53, 0x24, 0x4e, 0x2a, 0x13, 0xf, 0x29, 0x58, 0x77, 0x3b, 0x3]
misty_s9 = [0xa7, 0xef, 0xa1, 0x17b, 0x187, 0x14e, 0x9, 0x152, 0x26, 0xe2, 0x30, 0x166, 0x1c4, 0x181, 0x5a, 0x18d, 0xb7, 0xfd, 0x93, 0x14b, 0x19f, 0x154, 0x33, 0x16a, 0x132, 0x1f4, 0x106, 0x52, 0xd8, 0x9f, 0x164, 0xb1, 0xaf, 0xf1, 0x1e9, 0x25, 0xce, 0x11, 0x0, 0x14d, 0x2c, 0xfe, 0x17a, 0x3a, 0x8f, 0xdc, 0x51, 0x190, 0x5f, 0x3, 0x13b, 0xf5, 0x36, 0xeb, 0xda, 0x195, 0x1d8, 0x108, 0xac, 0x1ee, 0x173, 0x122, 0x18f, 0x4c, 0xa5, 0xc5, 0x18b, 0x79, 0x101, 0x1e0, 0x1a7, 0xd4, 0xf0, 0x1c, 0x1ce, 0xb0, 0x196, 0x1fb, 0x120, 0xdf, 0x1f5, 0x197, 0xf9, 0x109, 0x59, 0xba, 0xdd, 0x1ac, 0xa4, 0x4a, 0x1b8, 0xc4, 0x1ca, 0x1a5, 0x15e, 0xa3, 0xe8, 0x9e, 0x86, 0x162, 0xd, 0xfa, 0x1eb, 0x8e, 0xbf, 0x45, 0xc1, 0x1a9, 0x98, 0xe3, 0x16e, 0x87, 0x158, 0x12c, 0x114, 0xf2, 0x1b5, 0x140, 0x71, 0x116, 0xb, 0xf3, 0x57, 0x13d, 0x24, 0x5d, 0x1f0, 0x1b, 0x1e7, 0x1be, 0x1e2, 0x29, 0x44, 0x9c, 0x1c9, 0x83, 0x146, 0x193, 0x153, 0x14, 0x27, 0x73, 0x1ba, 0x7c, 0x1db, 0x180, 0x1fc, 0x35, 0x70, 0xaa, 0x1df, 0x97, 0x7e, 0xa9, 0x49, 0x10c, 0x117, 0x141, 0xa8, 0x16c, 0x16b, 0x124, 0x2e, 0x1f3, 0x189, 0x147, 0x144, 0x18, 0x1c8, 0x10b, 0x9d, 0x1cc, 0x1e8, 0x1aa, 0x135, 0xe5, 0x1b7, 0x1fa, 0xd0, 0x10f, 0x15d, 0x191, 0x1b2, 0xec, 0x10, 0xd1, 0x167, 0x34, 0x38, 0x78, 0xc7, 0x115, 0x1d1, 0x1a0, 0xfc, 0x11f, 0xf6, 0x6, 0x53, 0x131, 0x1a4, 0x159, 0x99, 0x1f6, 0x41, 0x3d, 0xf4, 0x11a, 0xad, 0xde, 0x1a2, 0x43, 0x182, 0x170, 0x105, 0x65, 0x1dc, 0x123, 0xc3, 0x1ae, 0x31, 0x4f, 0xa6, 0x14a, 0x118, 0x17f, 0x175, 0x80, 0x17e, 0x198, 0x9b, 0x1ef, 0x16f, 0x184, 0x112, 0x6b, 0x1cb, 0x1a1, 0x3e, 0x1c6, 0x84, 0xe1, 0xcb, 0x13c, 0xea, 0xe, 0x12d, 0x5b, 0x1f7, 0x11e, 0x1a8, 0xd3, 0x15b, 0x133, 0x8c, 0x176, 0x23, 0x67, 0x7d, 0x1ab, 0x13, 0xd6, 0x1c5, 0x92, 0x1f2, 0x13a, 0x1bc, 0xe6, 0x100, 0x149, 0xc6, 0x11d, 0x32, 0x74, 0x4e, 0x19a, 0xa, 0xcd, 0x1fe, 0xab, 0xe7, 0x2d, 0x8b, 0x1d3, 0x1d, 0x56, 0x1f9, 0x20, 0x48, 0x1a, 0x156, 0x96, 0x139, 0x1ea, 0x1af, 0xee, 0x19b, 0x145, 0x95, 0x1d9, 0x28, 0x77, 0xae, 0x163, 0xb9, 0xe9, 0x185, 0x47, 0x1c0, 0x111, 0x174, 0x37, 0x6e, 0xb2, 0x142, 0xc, 0x1d5, 0x188, 0x171, 0xbe, 0x1, 0x6d, 0x177, 0x89, 0xb5, 0x58, 0x4b, 0x134, 0x104, 0x1e4, 0x62, 0x110, 0x172, 0x113, 0x19c, 0x6f, 0x150, 0x13e, 0x4, 0x1f8, 0x1ec, 0x103, 0x130, 0x4d, 0x151, 0x1b3, 0x15, 0x165, 0x12f, 0x14c, 0x1e3, 0x12, 0x2f, 0x55, 0x19, 0x1f1, 0x1da, 0x121, 0x64, 0x10d, 0x128, 0x1de, 0x10e, 0x6a, 0x1f, 0x68, 0x1b1, 0x54, 0x19e, 0x1e6, 0x18a, 0x60, 0x63, 0x9a, 0x1ff, 0x94, 0x19d, 0x169, 0x199, 0xff, 0xa2, 0xd7, 0x12e, 0xc9, 0x10a, 0x15f, 0x157, 0x90, 0x1b9, 0x16d, 0x6c, 0x12a, 0xfb, 0x22, 0xb6, 0x1fd, 0x8a, 0xd2, 0x14f, 0x85, 0x137, 0x160, 0x148, 0x8d, 0x18c, 0x15a, 0x7b, 0x13f, 0x1c2, 0x119, 0x1ad, 0xe4, 0x1bb, 0x1e1, 0x5c, 0x194, 0x1e5, 0x1a6, 0xf8, 0x129, 0x17, 0xd5, 0x82, 0x1d2, 0x16, 0xd9, 0x11b, 0x46, 0x126, 0x168, 0x1a3, 0x7f, 0x138, 0x179, 0x7, 0x1d4, 0xc2, 0x2, 0x75, 0x127, 0x1cf, 0x102, 0xe0, 0x1bf, 0xf7, 0xbb, 0x50, 0x18e, 0x11c, 0x161, 0x69, 0x186, 0x12b, 0x1d7, 0x1d6, 0xb8, 0x39, 0xc8, 0x15c, 0x3f, 0xcc, 0xbc, 0x21, 0x1c3, 0x61, 0x1e, 0x136, 0xdb, 0x5e, 0xa0, 0x81, 0x1ed, 0x40, 0xb3, 0x107, 0x66, 0xbd, 0xcf, 0x72, 0x192, 0x1b6, 0x1dd, 0x183, 0x7a, 0xc0, 0x2a, 0x17d, 0x5, 0x91, 0x76, 0xb4, 0x1c1, 0x125, 0x143, 0x88, 0x17c, 0x2b, 0x42, 0x3c, 0x1c7, 0x155, 0x1bd, 0xca, 0x1b0, 0x8, 0xed, 0xf, 0x178, 0x1b4, 0x1d0, 0x3b, 0x1cd]


In [ ]:
def precomp_multab(n, poly):
    tab = {}
    for a, b in product(range(2**n), repeat=2):
        key = a, b
        res = 0
        while a:
            if a & 1:
                res ^= b
            a >>= 1
            b <<= 1
            if b >> 4:
                b ^= poly
        tab[key] = res
    return tab


class SSB16:
    sbox = NotImplemented
    poly = None
    n = r = m = 16

    def __init__(self):
        if self.poly:
            self.MULTAB = precomp_multab(n=4, poly=self.poly)
        self.SB = self.compute(self.sub)
        self.MC = self.compute(self.mix)
        self.SBMC = self.compute(self.sub, self.mix)
        self.MCSB = self.compute(self.mix, self.sub)
        self.SBMC

    def get_keys(self):
        return list(range(2**self.r))

    def get_part1(self):
        return self.SBMC

    def get_part2(self):
        return self.SB

    def mix(self, a, b, c, d):
        raise NotImplementedError()

    def sub(self, a, b, c, d):
        return tuple(map(self.sbox.__getitem__, (a, b, c, d)))

    def addk(self, *st, key=None):
        key = [
            key >> 12,
            (key >> 8) & 0xf,
            (key >> 4) & 0xf,
            (key >> 0) & 0xf,
        ]
        return tuple(vi ^ ki for vi, ki in zip(st, key))

    def compute(self, *funcs):
        s = []
        for a, b, c, d in product(range(16), repeat=4):
            for func in funcs:
                a, b, c, d = func(a, b, c, d)
            s.append((a << 12) | (b << 8) | (c << 4) | d)
        assert sorted(s) == list(range(2**16))
        return s

    def get_full(self, key):
        return self.compute(self.sub, lambda *st: self.addk(*st, key=key), self.mix, self.sub)
        # return self.compute(
        #     self.get_part1(),
        #     lambda *st: self.addk(*st, key=key),
        #     self.get_part2(),
        # )


class SSB_Midori64(SSB16):
    # same as MANTIS, CRAFT
    sbox = 0xc, 0xa, 0xd, 0x3, 0xe, 0xb, 0xf, 0x7, 0x8, 0x9, 0x1, 0x5, 0x0, 0x2, 0x4, 0x6

    def mix(self, a, b, c, d):
        t = a ^ b ^ c ^ d
        a ^= t
        b ^= t
        c ^= t
        d ^= t
        return a, b, c, d


class SSB_LED(SSB16):
    # same as PRESENT
    sbox = 0xC, 0x5, 0x6, 0xB, 0x9, 0x0, 0xA, 0xD, 0x3, 0xE, 0xF, 0x8, 0x4, 0x7, 0x1, 0x2
    poly = 0x13

    def mix(self, a, b, c, d):
        for i in range(4):
            tmp = (
                self.MULTAB[a, 4] ^ self.MULTAB[b, 1]
                ^ self.MULTAB[c, 2] ^ self.MULTAB[d, 2]
            )
            a, b, c, d = b, c, d, tmp
        return a, b, c, d


class SSB_SKINNY64(SSB16):
    sbox = 0xc, 0x6, 0x9, 0x0, 0x1, 0xa, 0x2, 0xb, 0x3, 0x8, 0x5, 0xd, 0x4, 0xe, 0x7, 0xf

    def mix(self, a, b, c, d):
        b ^= c
        c ^= a
        d ^= c
        a, b, c, d = d, a, b, c
        return a, b, c, d

    def get_keys(self):
        return (
            # constant affected (column 3?)
            [(a << 8) | 0x20 for a in range(16 * 16)] +
            # constant not affected 
            [(a << 8) | 0x00 for a in range(16 * 16)]
        )
            

class MISTY_FI:
    r = n = m = 16

    def __init__(self):
        self.part1 = []
        for l, r in product(range(2**9), range(2**7)):
            l = misty_s9[l]
            l ^= r  # zero-extend 7->9

            l, r = r, l

            l = misty_s7[l]
            l ^= r & 127  # truncate 9->7

            y = (l << 9) | r
            self.part1.append(y)
        assert sorted(self.part1) == sorted(range(2**16))

        self.part2 = []
        for l, r in product(range(2**7), range(2**9)):
            l, r = r, l

            l = misty_s9[l]
            l ^= r  # zero-extend 7->9

            l, r = r, l
            y = (l << 9) | r
            self.part2.append(y)
        assert sorted(self.part2) == sorted(range(2**16))

    def get_keys(self):
        return list(range(2**16))

    def get_part1(self):
        return self.part1

    def get_part2(self):
        return self.part2

    def get_full(self, key):
        res = []
        for x in range(2**16):
            x = self.part1[x]
            x = x ^ key
            x = self.part2[x]
            res.append(x)
        return res

In [ ]:
#S = SSB_SKINNY64()
#S = SSB_Midori64()
#S = SSB_LED()
S = MISTY_FI()
#key = choice(S.get_keys())
key = 0
#print("key = 0x%04x" % key)
SuperSbox = S.get_full(key=key)
assert sorted(SuperSbox) == list(range(2**16))

name = type(S).__name__ + '_' + str(key)
print(name)

In [ ]:
S = SuperSbox
n = 16
Alg = Algorithm(S,n,n)
print("init done")

In [ ]:
lengths = []
mx = 0
while True:
    res = GreedyRunHeap(Alg)
    l = len(res)
    lengths.append(l)
    if l >= mx:
        mx = l
        create_file(res,name,n)
        print(mx,len(lengths))